# Failure-Inducing Changes

"Yesterday, my program worked. Today, it does not. Why?" In debugging, as elsewhere in software development, code keeps on changing. Thus, it can happen that a piece of code that yesterday was working perfectly, today no longer runs – because we (or others) have made some changes to it that cause it to fail. The good news is that for debugging, we can actually _exploit_ this version history to narrow down _the changes that caused the failure_ – be it by us or by others.

In [ ]:
from bookutils import YouTubeVideo
# YouTubeVideo("w4u5gCgPlmg")

**Prerequisites**

* You should have read the [Chapter on Delta Debugging](DeltaDebugger.ipynb).
* Knowledge on version control systems (notably git) will be useful.

In [ ]:
import bookutils

In [ ]:
from bookutils import quiz, print_file, print_content

## Synopsis
<!-- Automatically generated. Do not edit. -->

To [use the code provided in this chapter](Importing.ipynb), write

```python
>>> from debuggingbook.ChangeDebugger import <identifier>
```

and then make use of the following features.


_For those only interested in using the code in this chapter (without wanting to know how it works), give an example.  This will be copied to the beginning of the chapter (before the first section) as text with rendered input and output._

You can use `int_fuzzer()` as:

```python
>>> print(int_fuzzer())
76.5
```


## Changes and Bugs

When you develop software, it is unlikely that you will be able to produce a fully working piece of software right from the beginning – hence the need for debugging. It is just as unlikely, though, that your software will stay _unchanged_ forever. Evolving requirements, the introduction of new technology, changes in the environment all induce software changes – and every such change brings the risk of introducing new bugs.

To detect such bugs introduced by changes, systematic (and best automatic) _testing_, notably _regression testing_, can be a big help; in fact, the more comprehensive and the more automated your testing strategy is, the more it actually _enables_ evolving your software, because every new test lowers the risk of a change introducing a bug. Extensive testing is what enables agile development – and we're happy to point to our [sibling book on test generation](https://www.fuzzingbook.org) to give you some inspiration on how to do this.

However, a test can only _detect_ failures, not _fix_ them. The more you change, the more you may need to fix, too. The good news is that there are a number of debugging techniques, manual and automated, that can actually _exploit_ the presence of older, working software versions to effectively narrow down the causes of failure in a new, failing software.

## A Version History

The fundamental prerequisite for exploiting older, working software versions is to have such older, working versions in the first place. If your software _always_ failed, then you will have to resort to conventional debugging. But if there is an older _working_ version, you can make use of it.

We assume that you do have a _version repository_ such as git or SVN, which you use to organize software development and keep older software versions.  (If you do _not_ use version control for your project, you are in [debugging hell](Intro_Debugging.ipynb). Go and set it up now, and come back once you're done.)

If you have a version history, an older working version, and a new failing version, your situation is roughly as depicted in this diagram:

In [ ]:
from graphviz import Digraph, nohtml

In [ ]:
from IPython.display import display

In [ ]:
# ignore
PASS = "✔"
FAIL = "✘"

PASS_COLOR = 'darkgreen'  # '#006400' # darkgreen
FAIL_COLOR = 'red4'  # '#8B0000' # darkred

STEP_COLOR = 'peachpuff'
FONT_NAME = 'Raleway'

In [ ]:
# ignore
def graph(comment="default"):
    return Digraph(name='', comment=comment, graph_attr={'rankdir': 'LR'},
        node_attr={'style': 'filled',
                   'shape': 'box',
                   'fillcolor': STEP_COLOR,
                   'fontname': FONT_NAME},
        edge_attr={'fontname': FONT_NAME})

In [ ]:
# ignore
VERSIONS = 8
state_machine = graph()
state_machine.edge(f'v1 {PASS}', 'v2')
for version_number in range(2, VERSIONS - 1):
    state_machine.edge(f'v{version_number}', f'v{version_number + 1}')
state_machine.edge(f'v{VERSIONS - 1}', f'v{VERSIONS} {FAIL}')

In [ ]:
# ignore
display(state_machine)

Somewhere between the old version "v1" ("yesterday") and the current version "v8" ("today"), the software stopped working. But when exactly? And which change was it that caused the failure?

You may think that this is an easy task: We simply manually test one version after another, thus determining the exact version that first failed. However, 

* this can take a long time (notably in the presence of dozens or even hundreds of versions);
* this may still leave you with _dozens of changes_ applied from one version to another; and
* this is actually a task that can be _fully automated_.

And these "automated" debugging techniques are what we explore in this chapter.

## A Version History

As our ongoing example, we start with creating a little version history, using the git version management system. We follow the evolution of the `remove_html_markup()` versions from [the introduction to debugging](Intro_Debugging.ipynb) and [the chapter on assertions](Assertions.ipynb).

### Create a Working Directory

We start with creating a working folder (aptly named `my_project`) in which we will do our work. (Note: should you have a folder of that name, it will be deleted and re-initialized).

In [ ]:
PROJECT = 'my_project'

In [ ]:
import os
import shutil

In [ ]:
try:
    shutil.rmtree(PROJECT)
except FileNotFoundError:
    pass
os.mkdir(PROJECT)

We choose the project folder as our working directory. Any file we create will be created in that folder.

In [ ]:
import sys

In [ ]:
sys.path.append(os.getcwd())
os.chdir(PROJECT)

### Initialize Git

We set up a local Git repository in our local project folder.

In [ ]:
!git init

In [ ]:
!git config advice.detachedHead False

We are now ready to commit our first version. Here's the initial definition of `remove_html_markup()` from [the introduction to debugging](Intro_Debugging.ipynb).

In [ ]:
def remove_html_markup(s):
    tag = False
    out = ""

    for c in s:
        if c == '<':    # start of markup
            tag = True
        elif c == '>':  # end of markup
            tag = False
        elif not tag:
            out = out + c

    return out

The function `write_source()` takes a function `fun` and writes its source code into a file of the same name – in our case, `remove_html_markup.py`:

In [ ]:
import inspect

In [ ]:
def write_source(fun, filename=None):
    if filename is None:
        filename = fun.__name__ + '.py'
    with open(filename, 'w') as fh:
        fh.write(inspect.getsource(fun))

Here is `write_source()` in action:

In [ ]:
write_source(remove_html_markup)

In [ ]:
print_file('remove_html_markup.py')

With `git add` and `git commit`, we add the file to our version repository. The `-m` option defines a _message_ for the commit; this is how we (and potential co-workers_ can later retrieve information on what has changed, and why. (The messages we use here are deliberately kept short.)

In [ ]:
!git add remove_html_markup.py

In [ ]:
!git commit -m "First version"

Let us now take the second (buggy) version of `remove_html_markup()` and again write this into our file, thus simulating changing the source code from the first version to the new version:

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

We can inspect the differences between the previously committed version and the current one. Lines prefixed with `+` are added; lines prefixed with `-` are deleted.

In [ ]:
!git diff remove_html_markup.py

We can now commit this second version, adding it to our repository.

In [ ]:
!git commit -m "Second version" remove_html_markup.py

We create a few more revisions.

### Excursion: More Revisions

We use the additional definitions for `remove_html_markup()` from [the introduction to debugging](Intro_Debugging.ipynb) as additional versions.

These also include "debugging" versions with enabled logging statements, as well as "tentative" versions that may or may not fix the discussed issues. In a real version history, such transient versions would typically not show up – or at least not be made available to co-workers.

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        print("c =", repr(c), "tag =", tag, "quote =", quote)

        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Third version (with debugging output)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<':  # and not quote:
            tag = True
        elif c == '>':  # and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Fourth version (clueless)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        assert not tag  # <=== Just added

        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Fifth version (with assert)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            assert False  # <=== Just added
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Sixth version (with another assert)" remove_html_markup.py

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif (c == '"' or c == "'") and tag:  # <-- FIX
            quote = not quote
        elif not tag:
            out = out + c

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Seventh version (fixed)" remove_html_markup.py

### End of Excursion

Here comes the last version of `remove_html_markup()`, this one from [the chapter on assertions](Assertions.ipynb).

In [ ]:
def remove_html_markup(s):
    tag = False
    quote = False
    out = ""

    for c in s:
        if c == '<' and not quote:
            tag = True
        elif c == '>' and not quote:
            tag = False
        elif c == '"' or c == "'" and tag:
            quote = not quote
        elif not tag:
            out = out + c

    # postcondition
    assert '<' not in out and '>' not in out

    return out

In [ ]:
write_source(remove_html_markup)

In [ ]:
!git commit -m "Eighth version (with proper assertion)" remove_html_markup.py

We finally reached the "today" state with the latest version – and find that the latest version has an error. This should come to you as no surprise if you have read the earlier chapters. But if you haven't, you will find that when the argument `s` contains double quotes, these are stripped from the output:

In [ ]:
remove_html_markup('"foo"')

Consequently, this test assertion fails:

In [ ]:
from ExpectError import ExpectError

In [ ]:
with ExpectError():
    assert remove_html_markup('"foo"') == '"foo"'

Note that the failure does _not_ occur in the very first version, as introduced above. So the simple question is:

* What is the change that cause the failure?

## Accessing Versions

We can look up the individual versions.

In [ ]:
!git log --pretty=oneline

In [ ]:
import subprocess

In [ ]:
def get_output(command):
    result = subprocess.run(command, 
                            stdout=subprocess.PIPE,
                            universal_newlines=True)
    return result.stdout

In [ ]:
log = get_output(['git', 'log', '--pretty=oneline'])
print(log)

In [ ]:
versions = [line.split()[0] for line in log.split('\n') if line]
versions.reverse()

We can check out the first version:

In [ ]:
!git checkout {versions[0]}

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

... and the last one:

In [ ]:
!git checkout {versions[7]}

In [ ]:
print_file('remove_html_markup.py')

This is the version that no longer works.

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

## Manual Bisecting

Bisecting is a cool technique to identify which commit caused the failure.

In [ ]:
!git bisect start

In [ ]:
!git bisect good {versions[0]}

In [ ]:
!git bisect bad {versions[7]}

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

In [ ]:
!git bisect bad

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

In [ ]:
!git bisect bad

In [ ]:
print_file('remove_html_markup.py')

In [ ]:
exec(open('remove_html_markup.py').read())

In [ ]:
remove_html_markup('"foo"')

We got the failure-inducing change:

In [ ]:
!git bisect bad

... and this is what it does – introducing `quote` handling:

In [ ]:
!git diff HEAD^

In [ ]:
!git bisect view

In [ ]:
!git bisect reset

## Automatic Bisecting

We can write a test script to automate bisecting. Its return code indicates the test outcome.

In [ ]:
# ignore
open('test.py', 'w').write('''
#!/usr/bin/env python

from remove_html_markup import remove_html_markup
import sys

result = remove_html_markup('"foo"')
if result == '"foo"':
    sys.exit(0)  # good/pass
elif result == 'foo':
    sys.exit(1)  # bad/fail
else:
    sys.exit(125)  # unresolved
''');

In [ ]:
print_file('test.py')

Right now, we are in the "fail" state:

In [ ]:
!python ./test.py; echo $?

In [ ]:
!git bisect start

In [ ]:
!git bisect good {versions[0]}

In [ ]:
!git bisect bad {versions[7]}

Here comes the automatic part:

In [ ]:
!git bisect run python test.py

Again, we obtain the failure-inducing change:

In [ ]:
!git diff HEAD^

In [ ]:
!git bisect reset

## Computing and Applying Patches

Our commit consists of a number of changes. Can we break this down further? Delta Debugging (on changes) to the rescue!

For this, though, we first need means to compute and apply patches.

In [ ]:
version_1 = get_output(['git', 'show', 
                            f'{versions[0]}:remove_html_markup.py'])

In [ ]:
print_content(version_1, '.py')

In [ ]:
version_2 = get_output(['git', 'show', 
                            f'{versions[1]}:remove_html_markup.py'])

In [ ]:
print_content(version_2, '.py')

In [ ]:
!git diff {versions[0]} {versions[1]}

We use Google's [diff-match-patch library](https://github.com/google/diff-match-patch).

In [ ]:
from diff_match_patch import diff_match_patch

The `diff()` function computes a set of patches (changes, diffs) between the two texts `s1` and `s2`:

In [ ]:
def diff(s1, s2, mode='lines'):
    dmp = diff_match_patch()
    if mode == 'lines':
        (text1, text2, linearray) = dmp.diff_linesToChars(s1, s2)
        diffs = dmp.diff_main(text1, text2)
        dmp.diff_charsToLines(diffs, linearray)
        return dmp.patch_make(diffs)

    if mode == 'chars':
        diffs = dmp.diff_main(s1, s2)
        return dmp.patch_make(s1, diffs)

    raise ValueError("mode must be 'lines' or 'chars'")

In [ ]:
patches = diff(version_1, version_2)
patches

Here's how to inspect these patches:

In [ ]:
import urllib

In [ ]:
def patch_string(p):
    return urllib.parse.unquote(str(p).strip())

In [ ]:
for p in patches:
    print(patch_string(p))

Conversely, the `patch()` function applies patches.

In [ ]:
def patch(s, patches):
    dmp = diff_match_patch()
    text, success = dmp.patch_apply(patches, s)
    assert all(success)
    return text

If we apply _all_ patches, we get version 2:

In [ ]:
print_content(patch(version_1, patches), '.py')

In [ ]:
assert patch(version_1, patches) == version_2

Applying _no_ patch leaves the content unchanged.

In [ ]:
assert patch(version_1, []) == version_1

However, one can also apply _partial_ sets of patches:

In [ ]:
print(patch_string(patches[0]))

In [ ]:
print_content(patch(version_1, [patches[0]]))

In [ ]:
print_content(patch(version_1, [patches[1]]))

## Delta Debugging on Patches

We now have everything we need.

In [ ]:
from DeltaDebugger import DeltaDebugger

In [ ]:
def test_remove_html_markup(patches):
    new_version = patch(version_1, patches)
    exec(new_version, globals())
    assert remove_html_markup('"foo"') == '"foo"'

In [ ]:
test_remove_html_markup([])

In [ ]:
with ExpectError():
    test_remove_html_markup(patches)

In [ ]:
with DeltaDebugger() as dd:
    test_remove_html_markup(patches)

In [ ]:
reduced_patches = dd.min_args()['patches']

In [ ]:
for p in reduced_patches:
    print(urllib.parse.unquote(str(p)))

In [ ]:
print_content(patch(version_1, reduced_patches), '.py')

Can we narrow this down even further? Yes!

In [ ]:
pass_patches, fail_patches, diffs = (arg['patches'] for arg in dd.min_arg_diff())

In [ ]:
print_content(patch(version_1, pass_patches), '.py')

In [ ]:
print_content(patch(version_1, fail_patches), '.py')

and the difference is:

In [ ]:
for p in diffs:
    print(urllib.parse.unquote(str(p)))

We learn that a single change introduced the failure.

## A ChangeDebugger class

Let us put all this together in a single class.

In [ ]:
from DeltaDebugger import DeltaDebugger, NotFailingError

In [ ]:
class ChangeDebugger(DeltaDebugger):
    def __init__(self, pass_source, fail_source, **ddargs):
        super().__init__(**ddargs)
        self._pass_source = pass_source
        self._fail_source = fail_source
        self._patches = diff(pass_source, fail_source)

    def pass_source(self):
        return self._pass_source
    def fail_source(self):
        return self._fail_source
    def patches(self):
        return self._patches

In [ ]:
def test_remove_html_markup():
    assert remove_html_markup('"foo"') == '"foo"'    

In [ ]:
with ChangeDebugger(version_1, version_2) as cd:
    test_remove_html_markup()

In [ ]:
with ExpectError(AssertionError):
    cd.call()

In [ ]:
print_content(cd.pass_source(), '.py')

In [ ]:
print_content(cd.fail_source(), '.py')

In [ ]:
cd.patches()

Now for the testing:

In [ ]:
class ChangeDebugger(ChangeDebugger):
    def test_patches(self, patches):
        new_version = patch(self.pass_source(), patches)
        exec(new_version, globals())
        self.call()

In [ ]:
with ChangeDebugger(version_1, version_2, log=True) as cd:
    test_remove_html_markup()

In [ ]:
cd.function()

In [ ]:
cd.test_patches([])

In [ ]:
with ExpectError(AssertionError):
    cd.test_patches(cd.patches())

Here's where `ChangeDebugger` applies the `DeltaDebugging` functionality on its own `test_patches()` method:

In [ ]:
class ChangeDebugger(ChangeDebugger):
    def min_patches(self):
        patches = self.patches()
        with self:
            self.test_patches(patches)
        return tuple(p['patches'] for p in dd.min_arg_diff())

    def __repr__(self):
        pass_patches, fail_patches, diff_patches = self.min_patches()
        return "".join(urllib.parse.unquote(str(p)) for p in diff_patches)

In [ ]:
with ChangeDebugger(version_1, version_2) as cd:
    test_remove_html_markup()

In [ ]:
cd.patches()

In [ ]:
pass_patches, fail_patches, diffs = cd.min_patches()
diffs

In [ ]:
cd

Success!

Does this also work for longer change histories? Let's take the very first and the very last version.

In [ ]:
version_8 = get_output(['git', 'show', 
                            f'{versions[7]}:remove_html_markup.py'])

In [ ]:
with ChangeDebugger(version_1, version_8) as cd:
    test_remove_html_markup()

In [ ]:
for p in cd.patches():
    print(urllib.parse.unquote(str(p)))

Again, success!

In [ ]:
cd

What happens if the function does not fail? Then, the `DeltaDebugger` diagnosis parts take over.

In [ ]:
with ExpectError(NotFailingError):
    with ChangeDebugger(version_1, version_2) as cd:
        remove_html_markup("foo")

## Synopsis

This chapter introduces a class `ChangeDebugger` that automatically determines failure-inducing code changes.

### High-Level Interface

Given two source files `source_pass` and `source_fail`, where `failing_function()` raises an exception in `source_pass`, but not in `source_fail`, you can use `ChangeDebugger` as follows:

```python
with ChangeDebugger(source_1, source_2) as cd:
    failing_function()
cd
```

This will produce the failure-inducing change between `source_pass` and `source_fail`.

In [ ]:
print(version_1)

In [ ]:
print(version_2)

In [ ]:
with ChangeDebugger(version_1, version_2) as cd:
    test_remove_html_markup()
cd

A programmatic interface is also available. The method `min_patches()` returns a triple (`pass_patches`, `fail_patches`, `diffs`) where

* applying `pass_patches` causes the call to pass
* applying `fail_patches` causes the call to fail
* `diffs` is the (minimal) difference between the two.

In [ ]:
pass_patches, fail_patches, diffs = cd.min_patches()

In [ ]:
for p in diffs:
    print(urllib.parse.unquote(str(p)))

### Supporting Functions

`ChangeDebugger` relies on lower level `patch()` and `diff()` functions.

To apply patch objects on source code, use the `patch()` function. It takes a source code and a list of patches to be applied.

In [ ]:
print(patch(version_1, diffs))

Conversely, the `diff()` function computes patches between two texts. It returns a list of patch objects that can be applied on text.

In [ ]:
for p in diff(version_1, version_2):
    print(urllib.parse.unquote(str(p)))

The `ChangeDebugger` class uses [Delta Debugging](DeltaDebugger.ipynb) to determine minimal differences in patches applied.

## Lessons Learned

* _Lesson one_
* _Lesson two_
* _Lesson three_

## Next Steps

_Link to subsequent chapters (notebooks) here, as in:_

* [use _mutations_ on existing inputs to get more valid inputs](MutationFuzzer.ipynb)
* [use _grammars_ (i.e., a specification of the input format) to get even more valid inputs](Grammars.ipynb)
* [reduce _failing inputs_ for efficient debugging](Reducer.ipynb)


## Background

Cite \cite{Zeller1999} and earlier works.

## Exercises

_Close the chapter with a few exercises such that people have things to do.  To make the solutions hidden (to be revealed by the user), have them start with_

```
**Solution.**
```

_Your solution can then extend up to the next title (i.e., any markdown cell starting with `#`)._

_Running `make metadata` will automatically add metadata to the cells such that the cells will be hidden by default, and can be uncovered by the user.  The button will be introduced above the solution._

### Exercise 1: _Title_

_Text of the exercise_

In [ ]:
# Some code that is part of the exercise
pass

_Some more text for the exercise_

**Solution.** _Some text for the solution_

In [ ]:
# Some code for the solution
2 + 2

_Some more text for the solution_

### Exercise 2: _Title_

_Text of the exercise_

**Solution.** _Solution for the exercise_